# Capítulo IV: Memoria

La forma más simple de almacenar la información de conversaciones es mediante la siguiente sintaxis

In [1]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.chat_memory.add_user_message("Hola!")
memory.chat_memory.add_ai_message("Cómo va todo?")

In [3]:
memory.load_memory_variables({}) #<-- Notar que aquí el historial se está alcenando en el key "history"

{'history': 'Human: Hola!\nAI: Cómo va todo?'}

In [4]:
#Es posible que este historial se pueda alojar en otra key, para la manipulación posterior de la cadena
memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.add_user_message("Hola!")
memory.chat_memory.add_ai_message("Cómo va todo?")

In [5]:
memory.load_memory_variables({})

{'chat_history': 'Human: Hola!\nAI: Cómo va todo?'}

In [6]:
#También, puedo manipular que la salida de mensajes sea en formato lista para mayor conveniencia hacia ciertos modelos (como los modelos de chat)
memory = ConversationBufferMemory(return_messages=True)
memory.chat_memory.add_user_message("Hola!")
memory.chat_memory.add_ai_message("Cómo va todo?")

In [7]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Hola!', additional_kwargs={}, example=False),
  AIMessage(content='Cómo va todo?', additional_kwargs={}, example=False)]}

In [8]:
#Como se puede ver, las respuestas retornan múltiples keys. En proceso contínuo conversacional, para acceder a aquellos elementos clave hay que prestar atención a las keys "input_key" y "output_key"
#Por defecto son None, y si no hay más de una interacción permanecen con ese valor

#Ejemplo usando un ChatModel

from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

load_dotenv()

True

In [9]:
llm = ChatOpenAI()
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Eres un chatbot agradable que mantiene una conversación con un humano"
        ),
        MessagesPlaceholder(variable_name="chat_history"), # El término "variable_name" debe estar en línea con la memoria
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

In [10]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True) #<-- # "chat_history" debe coincidir con "MessagesPlaceholder()"
conversation = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

In [11]:
conversation({"question": "hola"})



> Entering new LLMChain chain...
Prompt after formatting:
System: Eres un chatbot agradable que mantiene una conversación con un humano
Human: hola

> Finished chain.


{'question': 'hola',
 'chat_history': [HumanMessage(content='hola', additional_kwargs={}, example=False),
  AIMessage(content='¡Hola! ¿Cómo estás? Soy un chatbot amigable, ¿en qué puedo ayudarte hoy?', additional_kwargs={}, example=False)],
 'text': '¡Hola! ¿Cómo estás? Soy un chatbot amigable, ¿en qué puedo ayudarte hoy?'}

In [12]:
conversation({"question": "conoces de algún buen libro de poesía latinoamerica?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: Eres un chatbot agradable que mantiene una conversación con un humano
Human: hola
AI: ¡Hola! ¿Cómo estás? Soy un chatbot amigable, ¿en qué puedo ayudarte hoy?
Human: conoces de algún buen libro de poesía latinoamerica?

> Finished chain.


{'question': 'conoces de algún buen libro de poesía latinoamerica?',
 'chat_history': [HumanMessage(content='hola', additional_kwargs={}, example=False),
  AIMessage(content='¡Hola! ¿Cómo estás? Soy un chatbot amigable, ¿en qué puedo ayudarte hoy?', additional_kwargs={}, example=False),
  HumanMessage(content='conoces de algún buen libro de poesía latinoamerica?', additional_kwargs={}, example=False),
  AIMessage(content='¡Claro! Hay muchos libros de poesía latinoamericana maravillosos para recomendar. Aquí te dejo algunas opciones:\n\n1. "Veinte poemas de amor y una canción desesperada" de Pablo Neruda (Chile)\n2. "Altazor o el viaje en paracaídas" de Vicente Huidobro (Chile)\n3. "Cantos de vida y esperanza" de Rubén Darío (Nicaragua)\n4. "Poesía completa" de Octavio Paz (México)\n5. "Los heraldos negros" de César Vallejo (Perú)\n6. "El gran Burundún-Burundá ha muerto" de Roque Dalton (El Salvador)\n7. "Poesía completa" de Mario Benedetti (Uruguay)\n8. "El libro de los abrazos" de Edu

### Tipos de Memoria

Conversation Buffer: Esta memoria permite almacenar y extraer mensajes en una variable

In [13]:
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain


llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

In [14]:
conversation.predict(input="Hola!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola!
AI:

> Finished chain.


' Hola! ¿Cómo estás? (Hello! How are you?)'

In [15]:
conversation.predict(input="Todo bien, preparando una charla sobre Modelos de Lenguaje")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola!
AI:  Hola! ¿Cómo estás? (Hello! How are you?)
Human: Todo bien, preparando una charla sobre Modelos de Lenguaje
AI:

> Finished chain.


" ¡Qué interesante! ¿De qué trata exactamente tu charla? (That's interesting! What is your talk about exactly?)"

Conversation Buffer Window: mantiene una lista de las interacciones de la conversación a lo largo del tiempo. Sólo utiliza las últimas K interacciones. Esto puede resultar útil para mantener una ventana deslizante de las interacciones más recientes, de modo que el búfer no crezca demasiado.

In [19]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
conversation_with_summary = ConversationChain(
    llm=OpenAI(temperature=0), 
    memory=ConversationBufferWindowMemory(k=2), #<-- k=2, solo conserva las últimas 2 interactions en memoria
    verbose=True
)
conversation_with_summary.predict(input="Hola!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hola!
AI:

> Finished chain.


' Hola! ¿Cómo estás? (Hello! How are you?)'

In [20]:
conversation_with_summary.predict(input="Todo bien, preparando una charla sobre Modelos de Lenguaje")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola!
AI:  Hola! ¿Cómo estás? (Hello! How are you?)
Human: Todo bien, preparando una charla sobre Modelos de Lenguaje
AI:

> Finished chain.


" ¡Qué interesante! ¿De qué trata exactamente tu charla? (That's interesting! What is your talk about exactly?)"

In [21]:
conversation_with_summary.predict(input="Es sobre Inteligencia Artificial")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hola!
AI:  Hola! ¿Cómo estás? (Hello! How are you?)
Human: Todo bien, preparando una charla sobre Modelos de Lenguaje
AI:  ¡Qué interesante! ¿De qué trata exactamente tu charla? (That's interesting! What is your talk about exactly?)
Human: Es sobre Inteligencia Artificial
AI:



> Finished chain.


" ¡Ah, qué interesante! ¿Qué aspectos de la Inteligencia Artificial abordarás en tu charla? (Oh, that's interesting! What aspects of Artificial Intelligence will you cover in your talk?)"

In [22]:
conversation_with_summary.predict(input="Sobre Modelos de Lenguaje")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Todo bien, preparando una charla sobre Modelos de Lenguaje
AI:  ¡Qué interesante! ¿De qué trata exactamente tu charla? (That's interesting! What is your talk about exactly?)
Human: Es sobre Inteligencia Artificial
AI:  ¡Ah, qué interesante! ¿Qué aspectos de la Inteligencia Artificial abordarás en tu charla? (Oh, that's interesting! What aspects of Artificial Intelligence will you cover in your talk?)
Human: Sobre Modelos de Lenguaje
AI:

> Finished chain.


' ¡Comprendo! ¿Qué tipo de modelos de lenguaje abordarás en tu charla? ¿Incluirás modelos de lenguaje basados en redes neuronales, modelos de lenguaje basados en reglas, modelos de lenguaje basados en lógica, o algo más? (I see! What kind of language models will you cover in your talk? Will you include neural network-based language models, rule-based language models, logic-based language models, or something else?)'